### Initialize ellipsoids with DPD

In [51]:
import gsd, gsd.hoomd 
import hoomd
import mbuild as mb
import numpy as np
import scipy.stats
import unyt as u
import warnings
from flowermd.base import Pack,Lattice, Simulation,BaseHOOMDForcefield,Polymer
from flowermd.base.system import System
from flowermd.library import EllipsoidForcefield, EllipsoidChain
from flowermd.utils import get_target_box_number_density
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
from scipy.spatial.distance import pdist
warnings.filterwarnings('ignore')

In [52]:
class EllipsoidChainRand(Polymer):
    def __init__(
        self,
        lengths,
        num_mols,
        lpar,
        bead_mass,
        bond_L=0.01,
        name="ellipsoid_chain",
    ):
        self.bead_mass = bead_mass
        self.lpar = lpar
        self.bond_L = bond_L
        self.bead_constituents_types = ["X", "A", "T", "T"]
        super(EllipsoidChainRand, self).__init__(
            lengths=lengths, num_mols=num_mols, name=name
        )

    def _build(self, length):
        bead = mb.Compound(name="ellipsoid")
        center = mb.Compound(pos=(0, 0, 0), name="X", mass=self.bead_mass / 4)
        head = mb.Compound(
            pos=(0, 0, self.lpar + (self.bond_L / 2)),
            name="A",
            mass=self.bead_mass / 4,
        )
        tether_head = mb.Compound(
            pos=(0, 0, self.lpar), name="T", mass=self.bead_mass / 4
        )
        tether_tail = mb.Compound(
            pos=(0, 0, -self.lpar), name="T", mass=self.bead_mass / 4
        )
        bead.add([center, head, tether_head, tether_tail])
        bead.add_bond([center, head])

        chain = mb.Compound()
        last_bead = None
        buffer = self.lpar + (self.bond_L / 2)
        density = 0.5
        N=50
        L = np.cbrt(N / density)
        for i in range(length):
            translate_by = np.random.uniform(low=-((L/2)-buffer), high=(L/2)-buffer, size=(3,))#adding random bead shift
            print(translate_by)
            this_bead = mb.clone(bead)
            this_bead.translate(by=translate_by)
            chain.add(this_bead)
            if last_bead:
                chain.add_bond([this_bead.children[0], last_bead.children[1]])
                chain.add_bond([this_bead.children[3], last_bead.children[2]])
            last_bead = this_bead
        chain.name = f"{self.name}_{length}mer"

        return chain

class DPD_FF(BaseHOOMDForcefield):
    def __init__(
        self,
        epsilon,
        lpar,
        lperp,
        A,
        gamma,
        kT,
        r_cut,
        angle_k=None,
        angle_theta0=None,
        bond_k=100,
        bond_r0=0.01,
    ):
        self.epsilon = epsilon
        self.lpar=lpar
        self.lperp=lperp
        self.gamma = gamma
        self.A = A
        self.kT = kT
        self.r_cut = r_cut
        self.angle_k = angle_k
        self.angle_theta0 = angle_theta0
        self.bond_k = bond_k
        self.bond_r0 = bond_r0
        hoomd_forces = self._create_forcefield()
        super(DPD_FF, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        forces = []
        # Bonds
        bond = hoomd.md.bond.Harmonic()
        bond.params["T-T"] = dict(k=self.bond_k, r0=self.bond_r0)
        bond.params["A-X"] = dict(k=0, r0=0)
        forces.append(bond)
        # Angles
        if all([self.angle_k, self.angle_theta0]):
            angle = hoomd.md.angle.Harmonic()
            angle.params["X-A-X"] = dict(k=self.angle_k, t0=self.angle_theta0)
            angle.params["A-X-A"] = dict(k=0, t0=0)
            forces.append(angle)
        # DPD Pairs
        nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["body"])
        dpd = hoomd.md.pair.DPD(nlist=nlist,kT=self.kT,default_r_cut=self.r_cut)
        dpd.params[("X", "X")] = dict(A=self.A, gamma=self.gamma)
        # Add zero pairs
        for pair in [
            ("R", "R"),
            ("T", "T"),
            ("T", "R"),
            ("A", "A"),
            ("A", "X"),
            ("A", "T"),
            ("A", "R"),
            ("X", "R"),
            ("X", "T"),
        ]:
            dpd.params[pair] = dict(A=0,gamma=0.1)
            dpd.params[pair].r_cut = 0.0
        forces.append(dpd)
        return forces

#write new system class to skip pack and shrink step
class RandomSystem(System):
    def __init__(self, molecules, base_units=dict(), buffer=1.00):
        self.buffer = buffer
        super(RandomSystem, self).__init__(
            molecules=molecules, base_units=base_units
        )
    def _build_system(self):
        chain = self.all_molecules
        L = 5.85 #need to figure how to get this from molecules #TODO
        box = mb.Box(lengths=np.array([L] * 3) * self.buffer)
        comp = mb.Compound()
        comp.add(chain)
        comp.box = box
        return comp

def gen_ellipse_chain(chain_length = 5, N = 1, long_axis = 1, minor_axis = 0.5):
    ellipsoid_chain = EllipsoidChainRand(lengths=chain_length,num_mols=N,lpar=1.0,bead_mass=1.0)
    ff = DPD_FF(epsilon=1.0,lpar=long_axis,lperp=minor_axis,A=100,gamma=1.0,kT=2.0,r_cut=2.0,bond_k=100,bond_r0=0.01,angle_k=30,angle_theta0=1.9)
    system = RandomSystem(molecules=ellipsoid_chain)
    #system.to_gsd('test.gsd')
    #gsd_path=('ellipsoid-test.gsd')
    rigid_frame, rigid = create_rigid_ellipsoid_chain(
        system.hoomd_snapshot,lpar=long_axis,lperp=minor_axis
    )
    ellipsoid_sim = Simulation(
        initial_state=rigid_frame,
        forcefield=ff.hoomd_forces,
        constraint=rigid,
        dt=0.0003,
        gsd_write_freq=int(100),
        gsd_file_name='trajectory.gsd',
        log_write_freq=int(100),
        log_file_name='log.txt')    
    ellipsoid_sim.save_restart_gsd()
    ellipsoid_sim.run_NVT(n_steps=1e4, kT=2.0, tau_kt=10*ellipsoid_sim.dt)
    ellipsoid_sim.flush_writers()
    
    s= ellipsoid_sim.state.get_snapshot()
    positions = s.particles.position[:chain_length]
    orientations = s.particles.orientation[:chain_length]
    return positions, orientations


In [54]:

positions,orientations = gen_ellipse_chain(chain_length = 5, N=1, long_axis = 1.0, minor_axis = 0.5)

*Warning*: angle.harmonic: specified K <= 0
*Warning*: angle.harmonic: specified t_0 <= 0


[-1.10122449 -1.01501857  0.37441967]
[-0.64375716 -0.94369142 -1.18368001]
[ 0.00153531 -1.08987651  0.6728673 ]
[ 1.04197266 -1.29305919 -0.1773392 ]
[-0.39836347  1.18241258 -0.97298625]
Initializing simulation state from a gsd.hoomd.Frame.
Step 100 of 10000; TPS: 783.53; ETA: 0.2 minutes
Step 200 of 10000; TPS: 1558.65; ETA: 0.1 minutes
Step 300 of 10000; TPS: 2327.11; ETA: 0.1 minutes
Step 400 of 10000; TPS: 3089.21; ETA: 0.1 minutes
Step 500 of 10000; TPS: 3843.76; ETA: 0.0 minutes
Step 600 of 10000; TPS: 4590.17; ETA: 0.0 minutes
Step 700 of 10000; TPS: 5325.18; ETA: 0.0 minutes
Step 800 of 10000; TPS: 6057.58; ETA: 0.0 minutes
Step 900 of 10000; TPS: 6784.21; ETA: 0.0 minutes
Step 1000 of 10000; TPS: 7506.77; ETA: 0.0 minutes
Step 1100 of 10000; TPS: 8222.33; ETA: 0.0 minutes
Step 1200 of 10000; TPS: 8933.22; ETA: 0.0 minutes
Step 1300 of 10000; TPS: 9637.7; ETA: 0.0 minutes
Step 1400 of 10000; TPS: 10338.44; ETA: 0.0 minutes
Step 1500 of 10000; TPS: 11034.2; ETA: 0.0 minutes
S

In [55]:
print(orientations)

[[-0.23849195  0.52056122  0.78567809  0.23419555]
 [ 0.66959475 -0.29917047  0.4311085   0.52562853]
 [ 0.89793777  0.259712    0.33583407  0.11607286]
 [ 0.68546274 -0.65098023  0.09712842 -0.31133845]
 [-0.01492999 -0.68936814 -0.26568322 -0.67376635]]


In [56]:
print(positions)

[[-1.70173217 -2.53356297 -2.44004679]
 [-1.54934459 -0.90357235 -2.6251603 ]
 [-0.69305098 -0.59401168 -1.26053283]
 [ 0.5885601  -0.23859246 -0.46789553]
 [ 2.25573041  1.11050631 -0.34308297]]
